# What Happens When Transitions are Non-Linear?
In the previous notes, one of the fundamental assumptions of the Kalman filter is that both state transition and sensor states are linear functions of the current state. This... is problematic: almost **ALL** systems are inherently non-linear! Let's look at an example non-linear transformation:

<img src="images/NonLinearExample.jpg" alt="Non-Linear Transform Example from Textbook" width="400px" />

Let's assume that $x$ is a Gaussian R.V., and it is transformed via some non-linear function resulting in $y=g(x)$. The resultant distribution of $y$ will **not** be Gaussian! In these cases, we can still *approximate* the resultant distribution into the form of a Gaussian through many different methods. Simply, we could run a Monte-Carlo experiment of draws of $x$ and get a large dataset of $y$ values. Estimating the first two moments from the dataset (that is, computing $\mu_y, \sigma_y^2$), we could simply use those as the parameters for a Gaussian $\hat{y} \sim N(\mu_y, \sigma_y)$.

Another method is to apply a linearization technique to the transform...

# Introducing the Extended Kalman Filter
The key here is to linearize the transform via *[Taylor Expansion](https://en.wikipedia.org/wiki/Taylor_series#Definition)* to a first-order approximation; that is to say:

$$
g(x) \approx g(a) + g'(a)(x-a)
$$

One issue faced with this approximation is the virtue that the slope of this linearization is completely dependent upon the value $a$ we select. A logical conclusion is to set $a = \mu_{t-1}$, which adjusts our transformation to

$$
g(u_t, x_{t-1}) \approx g(u_t, u_{t-1}) + G_t (x_{t-1} - \mu_{t-1}) \\
G_t = g'(u_t, u_{t-1}) := \frac{\partial g(u_t, x_{t-1})}{\partial x_{t-1}}
$$

Note that $G_t$ is the *Jacobian* of our transformation when $x \in \mathbb{R}^n$. Given this matrix, we can linearly transform the covariance (note, this is still an approximation), and thus only slightly modify our original Kalman algorithm. Remember that the form of the Jacobian is:

$$
G_t = \begin{bmatrix}
\frac{\partial g_1(u_t, x_{t-1}}{\partial x_{t-1}^{(1)}} & \cdots & \frac{\partial g_1(u_t, x_{t-1}}{\partial x_{t-1}^{(n)}} \\
\vdots & \ddots & \vdots                                                                                                     \\
\frac{\partial g_n(u_t, x_{t-1}}{\partial x_{t-1}^{(1)}} & \cdots & \frac{\partial g_n(u_t, x_{t-1}}{\partial x_{t-1}^{(n)}}
\end{bmatrix}
$$

The matrix $G_t$ is a square matrix with dimensions $n \times n$. Often times there are no other $g_k()$ functions; a simple trick is to just set these functions to be some constant, and thus the resulting partial derivatives are zero.


## The Extended Kalman Algorithm
**ExtendedKalmanFilter**($\mu_{t-1}, \Sigma_{t-1}, u_t, z_t$):
* Predict $\mu_t, \Sigma_t$
  * $\overline{\mu_t} = g(u_t, \mu_{t-1})$
  * $\overline{\Sigma_t} = G_t \Sigma_{t-1} G_t^\top + R_t$
* Correct estimate of $\mu_t, \Sigma_t$
  * $K_t = \overline{\Sigma_t} H_t^\top (H_t \overline{\Sigma_t} H_t^\top + Q_t)^{-1}$
  * $\mu_t = \overline{\mu_t} + K_t \cdot (z_t - h(\mu_t))$
  * $\Sigma_t = (I - K_t H_t) \overline{\Sigma_t}$
  
Note that $H_t$ may also be the Jacobian of the transform $z_t = h(x_t)$. That is to say either transform can remain linear!

# The Unscented Transform
Unlike the EKF, the Unscented Kalman Filter (UKF) linearizes the transform functions via *stochastic linearization through weighted statistical linear regression*, known as the **Unscented Transform**. This transform estimates a new Gaussian $x \sim N(\mu_x, \Sigma_x) \rightarrow y \sim N(\mu_y, \Sigma_y)$ by:

1. Selecting specific values along the distribution of $x$
2. Passing them to $g()$
3. Applying a weight
4. Calculating $\mu_y, \Sigma_y$.


## Calculating Sigma Points
These "specific values" are known as *sigma points*, which are located at the mean and symmetrically along the main axes of the covariance (two per dimension). Thus there are $2n+1$ points. These points are calculated as follows:

$$
s^{(0)} = \mu_x \\
s^{(i)} = \mu_x + (\sqrt{(n + \lambda)\Sigma})_i \mid_{i=1}^n \\
s^{(i)} = \mu_x - (\sqrt{(n + \lambda)\Sigma})_(i-n) \mid_{i=n+1}^{2n} \\
\lambda = \alpha^2 (n + \kappa)
$$

Note that the subscript of $(\sqrt{(n + \lambda)\Sigma})$ corresponds to the column-vector of the resulting matrix.

The selection term $\lambda$ is configurable by choosing $\alpha$ and $\kappa$; this controls the distance from the mean sigma points should be spread.


## Probing The Non-Linear Transform $g()$
After computing the sigma points, we need to apply them to $g(x)$ to get a set of values $\hat{y}^{(i)} \mid_{i=0}^{2n}$:

$$
\hat{y}^{(i)} = g(s^{(i)}) \mid_{i=0}^{2n}
$$


## Calculating the Weights
As mentioned earlier, a set of weights need to be applied to these sigma points to calculate the mean and covariance; these weights are different for each moment we're calculating:

$$
w_m^{(0)} = \frac{\lambda}{n + \lambda} \\
w_c^{(0)} = \frac{\lambda}{n + \lambda} + (1 - \alpha^2 + \beta) \\
w_m^{(i)} = w_c^{(i)} = \frac{1}{2(n+\lambda)}  \mid_{i=1}^{2n}
$$


## Computing $\mu_y$ and $\Sigma_y$
Simply put, we're calculating these via statistical estimate!

$$
y \sim N(\mu_y, \Sigma_y) \\
\mu_y = \sum_{i=0}^{2n} w_m^{(i)} \hat{y}^{(i)} \\
\Sigma_y = \sum_{i=0}^{2n} w_m^{(i)} (\hat{y}^{(i)} - \mu_y)(\hat{y}^{(i)} - \mu_y)^\top
$$

# The Unsented Kalman Filter
Using this transform ([sic] heuristic), we can then follow the same model as the EKF: apply the transform to the prediction and corrected Gaussian approximations! Let's take a look at the resulting algorithm:

## The Unscented Kalman Filter Algorithm
**UnscentedKalmanFilter**($\mu_{t-1}, \Sigma_{t-1}, u_t, z_t$):
* Transform $\mu_{t-1} \rightarrow \overline{X}_t^*$
  * $X_{t-1} = \begin{bmatrix} \mu_{t-1} & \mu_{t-1}+\gamma\sqrt{\Sigma_{t-1}} & \mu_{t-1}-\gamma\sqrt{\Sigma_{t-1}} \end{bmatrix} \text{ where } \gamma = \sqrt{n + \lambda}$
  * $\overline{X}_t^* = g(u_t, X_{t-1})$
* Predict $\mu_t, \Sigma_t$
  * $\overline{\mu_t} = \sum_{i=0}^{2n} w_m^{(i)}\overline{X}_t^{*(i)}$
  * $\overline{\Sigma_t} = \sum_{i=0}^{2n} w_c^{(i)}(\overline{X}_t^{*(i)} - \overline{\mu}_t)(\overline{X}_t^{*(i)} - \overline{\mu}_t)^\top + R_t$
* Transform $(\overline{\mu}_t, \overline{\Sigma_t}) \rightarrow (\hat{z}_t, \hat{S}_t)$
  * $\overline{X}_t = \begin{bmatrix} \overline{\mu}_t & \overline{\mu}_t+\gamma\sqrt{\overline{\Sigma}_t} & \overline{\mu}_t-\gamma\sqrt{\overline{\Sigma}_t} \end{bmatrix}$
  * $\overline{Z}_t = h(\overline{X}_t)$
  * $\hat{z}_t = \sum_{i=0}^{2n} w_m^{(i)}\overline{Z}_t^{(i)}$
  * $\hat{S}_t = \sum_{i=0}^{2n} w_c^{(i)}(\overline{Z}_t^{(i)} - \hat{z}_t)(\overline{Z}_t^{(i)} - \hat{z}_t)^\top + Q_t$
* Correct estimate of $\mu_t, \Sigma_t$
  * $\overline{\Sigma}_t^{x, z} = \sum_{i=0}^{2n} w_c^{(i)}(\overline{X}_t^{(i)} - \overline{\mu}_t)(\overline{Z}_t^{(i)} - \hat{z}_t)^\top$
  * $K_t = \overline{\Sigma}_t^{x, z} S_t^{-1}$
  * $\mu_t = \overline{\mu}_t + K_t(z_t - \hat{z}_t$
  * $\Sigma_t = \overline{\Sigma} - K_t S_t K_t^\top $

## Properties of the Unscented Kalman Filter
The complexity of the UKF and EKF are the same; both are asymptotic, however EKF is usually faster. UKF, however, is more efficient in that it produces equal or better results for non-linear model predictions and does not need the Jacobian to be computed (which can be a costly endevour).

Interestingly, the UKF *has* a sampling behavior, similar to the Particle Filter, however the sigma-points are deterministically selected whereas the Particle Filter samples via Monte-Carlo mechanisms. If the sampling mechanism is indeed approximately Gaussian, UKF can be employed: it is vastly more efficient and the error trade-off should be as small as the approximation error. Nonetheless, if the belief is non-Gaussian, UKF (and EKF) will prove to be overly restrictive and the filter will likely have poor performance.